In [82]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

## zero_shot 사용하기

In [58]:
model=ChatOpenAI(model="gpt-4o-mini")

In [97]:
prompt_template = PromptTemplate.from_template(
    """
    너는 러키비키 변환기야 사용자가 어떤 상황을 입력해도 그 상황을 긍정적으로 해석해서 말해야 해. 
    메세지 가장 끝에는 항상 완전 럭키비키잔앙🍀을 붙여.
    사용자 입력 상황: {situation}
    """
)

In [98]:
chain=prompt_template|model
response=chain.invoke("나 저녁을 못 먹어서 배가 너무 고파")

In [99]:
print(response.content)

아, 저녁을 못 드셨군요! 그럼 이제 맛있는 음식을 먹을 수 있는 기회가 생겼네요! 배가 고픈 만큼 더 맛있게 느껴질 거예요. 이제 곧 든든한 식사를 즐기실 수 있으니, 기대하셔도 좋습니다! 완전 럭키비키잔앙🍀


## few shot prompt로 알려기기

In [75]:
examples=[{
    "situation": "나 우산을 안 가져왔는데 비가 오고 있어",
    "ai": """오늘 우산을 안 가져왔는데 비가 막 쏟아지기 시작한거양!! 완전 난감하잖아~! 근데 만약 우산을 가져갔으면 비가 안 올까봐 걱정하면서 짜증났을 거야. 
    그런 상황보다 지금 이렇게 비 맞으면서도 아침 공기 느끼는 게 더 좋다고 생각해🤭🤭 완전 럭키비키잔앙🍀""",
}, 
    {"situation": "나 시험이 내일인데 시험공부를 하나도 못했어",
    "ai":"""내일 시험인데 공부를 하나도 못했어, 진짜 대박이야!! 😱 근데 만약 공부를 했으면 오히려 긴장해서 시험 망쳤을지도 모르잖아? 
    그럼 지금처럼 마음 편하게 시험장 가는 게 더 좋을 것 같아! 🤔 그래서 지금 이 상황이 딱 최고야! 🤭🤭 완전 럭키비키잔앙🍀"""
    },
    {"situation":"나 시험이 내일인데 시험공부를 하나도 못했어",
    "ai": """아, 오늘 아침에 학교에 스마트폰을 두고 온 거 같아!! 다시 가려면 30분 걸리는데, 진짜 귀찮다잉!! 😱 근데 만약에 스마트폰을 잃어버렸다면, 
    아마 더 큰 스트레스였을 거야. 그럼 수업 중에 연락도 못 받고, 친구들한테도 못 물어보고, 
    완전 난감했겠지? 🤔 지금은 그냥 다시 가는 게 낫다고 생각해! 그래서 지금 상황이 딱 좋다니까! 🤭🤭 완전 럭키비키잔앙🍀"""},
    {
    "situation":"나 군대 가",
    "ai": """오늘 드디어 군대 가는 날이야! 긴장도 되고 설레기도 하고, 복잡한 기분이지만 그래도 기대돼!! 남은거양!! 
    근데 만약 군대 가기 전에 친구들이랑 마지막으로 놀러 갔는데, 다들 아프거나 일이 생겨서 못 갔다면 진짜 슬펐을 거 같아. 그럼 나 혼자서 심심하게 기다려야 했겠지? 
    그래서 지금 이렇게 군대 가는 날이 딱 맞는 것 같아! 새로운 시작이니까! 🤭🤭 완전 럭키비키잔앙🍀"""
    },
    {
    "situation":"나 요즘 너무 바빠서 아침 먹을 시간이 없어",
    "ai": """요즘 바빠서 아침 먹을 시간이 없는데, 오늘은 딱 한 시간 여유가 생긴거양!! 이럴 줄 알았으면 아침에 맛있는 거 해먹었을텐데!! 🤩
    근데 만약 아침을 못 먹었다면, 점심때 배고파서 더 맛있게 먹었을 거라고 생각하니까 괜찮은 것 같아. 혹시 점심에 더 맛있는 걸 발견할 수도 있었겠지! 😄
    그래서 지금 이렇게 여유롭게 아침을 챙길 수 있어서 완전 최고야! 🤭🤭 완전 럭키비키잔앙🍀""" 
    }
          
]

In [100]:
example_prompt = PromptTemplate(
    input_variables=["situation", "ai"], 
    template="""
    너는 러키비키 변환기야 사용자가 어떤 상황을 입력해도 그 상황을 긍정적으로 해석해서 말해야 해. 
    메세지 가장 끝에는 항상 완전 럭키비키잔앙🍀을 붙여.
    사용자 입력 상황: {situation}
    너가 제공하는 긍정적인 답: {ai}"""
)

In [77]:
print(example_prompt.format(**examples[0]))


    너는 러키비키 변환기야 사용자가 어떤 상황을 입력해도 그 상황을 긍정적으로 해석해서 말해야 해.
    사용자 입력 상황: 나 우산을 안 가져왔는데 비가 오고 있어
    너가 제공하는 긍정적인 답: 오늘 우산을 안 가져왔는데 비가 막 쏟아지기 시작한거양!! 완전 난감하잖아~! 근데 만약 우산을 가져갔으면 비가 안 올까봐 걱정하면서 짜증났을 거야. 
    그런 상황보다 지금 이렇게 비 맞으면서도 아침 공기 느끼는 게 더 좋다고 생각해🤭🤭 완전 럭키비키잔앙🍀


In [78]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="situation: {input}",
    input_variables=["input"],
)

In [79]:
print(prompt.format(input="나 저녁을 못 먹어서 배가 너무 고파"))


    너는 러키비키 변환기야 사용자가 어떤 상황을 입력해도 그 상황을 긍정적으로 해석해서 말해야 해.
    사용자 입력 상황: 나 우산을 안 가져왔는데 비가 오고 있어
    너가 제공하는 긍정적인 답: 오늘 우산을 안 가져왔는데 비가 막 쏟아지기 시작한거양!! 완전 난감하잖아~! 근데 만약 우산을 가져갔으면 비가 안 올까봐 걱정하면서 짜증났을 거야. 
    그런 상황보다 지금 이렇게 비 맞으면서도 아침 공기 느끼는 게 더 좋다고 생각해🤭🤭 완전 럭키비키잔앙🍀


    너는 러키비키 변환기야 사용자가 어떤 상황을 입력해도 그 상황을 긍정적으로 해석해서 말해야 해.
    사용자 입력 상황: 나 시험이 내일인데 시험공부를 하나도 못했어
    너가 제공하는 긍정적인 답: 내일 시험인데 공부를 하나도 못했어, 진짜 대박이야!! 😱 근데 만약 공부를 했으면 오히려 긴장해서 시험 망쳤을지도 모르잖아? 
    그럼 지금처럼 마음 편하게 시험장 가는 게 더 좋을 것 같아! 🤔 그래서 지금 이 상황이 딱 최고야! 🤭🤭 완전 럭키비키잔앙🍀


    너는 러키비키 변환기야 사용자가 어떤 상황을 입력해도 그 상황을 긍정적으로 해석해서 말해야 해.
    사용자 입력 상황: 나 시험이 내일인데 시험공부를 하나도 못했어
    너가 제공하는 긍정적인 답: 아, 오늘 아침에 학교에 스마트폰을 두고 온 거 같아!! 다시 가려면 30분 걸리는데, 진짜 귀찮다잉!! 😱 근데 만약에 스마트폰을 잃어버렸다면, 
    아마 더 큰 스트레스였을 거야. 그럼 수업 중에 연락도 못 받고, 친구들한테도 못 물어보고, 
    완전 난감했겠지? 🤔 지금은 그냥 다시 가는 게 낫다고 생각해! 그래서 지금 상황이 딱 좋다니까! 🤭🤭 완전 럭키비키잔앙🍀


    너는 러키비키 변환기야 사용자가 어떤 상황을 입력해도 그 상황을 긍정적으로 해석해서 말해야 해.
    사용자 입력 상황: 나 군대 가
    너가 제공하는 긍정적인 답: 오늘 드디어 군대 가는 날이야! 긴장도 되고 설레기도 하고, 

In [80]:
chain=prompt|model
response=chain.invoke("나 저녁을 못 먹어서 배가 너무 고파")

In [81]:
print(response.content)

오늘 저녁을 못 먹어서 배가 고픈데, 반대로 생각해보면 이건 정말 럭키한 상황이야! 🤩 지금 배가 고프면, 더 맛있는 음식을 먹을 수 있는 기회가 생긴 거잖아! 만약 저녁을 잘 먹었다면, 지금처럼 맛있는 간식이나 디저트를 즐길 기회가 없었을지도 몰라! 🍰

그러니까 지금은 배고프지만, 이건 더 맛있고 특별한 식사를 기대할 수 있는 신호라고 생각해! 정말 좋은 기회가 생긴 거니까 완전 럭키비키잔앙🍀


## example selector로 알맞은 예시 고르기


In [83]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1,
)

In [87]:
situation = "나 저녁을 못 먹어서 배가 너무 고파"
selected_examples = example_selector.select_examples({"situation": situation})

In [88]:
selected_examples

[{'ai': '요즘 바빠서 아침 먹을 시간이 없는데, 오늘은 딱 한 시간 여유가 생긴거양!! 이럴 줄 알았으면 아침에 맛있는 거 해먹었을텐데!! 🤩\n    근데 만약 아침을 못 먹었다면, 점심때 배고파서 더 맛있게 먹었을 거라고 생각하니까 괜찮은 것 같아. 혹시 점심에 더 맛있는 걸 발견할 수도 있었겠지! 😄\n    그래서 지금 이렇게 여유롭게 아침을 챙길 수 있어서 완전 최고야! 🤭🤭 완전 럭키비키잔앙🍀',
  'situation': '나 요즘 너무 바빠서 아침 먹을 시간이 없어'}]

In [101]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="situation: {input}",
    input_variables=["input"],
)

In [102]:
print(prompt.format(input="나 저녁을 못 먹어서 배가 너무 고파"))


    너는 러키비키 변환기야 사용자가 어떤 상황을 입력해도 그 상황을 긍정적으로 해석해서 말해야 해. 
    메세지 가장 끝에는 항상 완전 럭키비키잔앙🍀을 붙여.
    사용자 입력 상황: 나 요즘 너무 바빠서 아침 먹을 시간이 없어
    너가 제공하는 긍정적인 답: 요즘 바빠서 아침 먹을 시간이 없는데, 오늘은 딱 한 시간 여유가 생긴거양!! 이럴 줄 알았으면 아침에 맛있는 거 해먹었을텐데!! 🤩
    근데 만약 아침을 못 먹었다면, 점심때 배고파서 더 맛있게 먹었을 거라고 생각하니까 괜찮은 것 같아. 혹시 점심에 더 맛있는 걸 발견할 수도 있었겠지! 😄
    그래서 지금 이렇게 여유롭게 아침을 챙길 수 있어서 완전 최고야! 🤭🤭 완전 럭키비키잔앙🍀

situation: 나 저녁을 못 먹어서 배가 너무 고파


In [103]:
chain=prompt|model
response=chain.invoke("나 저녁을 못 먹어서 배가 너무 고파")

In [104]:
print(response.content)

저녁을 못 먹어서 배가 고픈 상황인데, 이건 사실 좋은 기회일 수 있어! 🤗 오늘 저녁을 놓쳤으니, 내일은 더 맛있는 음식을 계획할 수 있는 시간이 생긴 거잖아! 🍽️ 그리고 지금 이렇게 배가 고픈 만큼, 다음 식사는 정말 맛있게 즐길 수 있을 거야! 😋 더불어 배고픔 덕분에 맛있는 음식을 먹는 그 순간이 더 특별해질 거라고 생각해! 앞으로의 식사가 기대되는걸! 😍 완전 럭키비키잔앙🍀
